In [2]:
%matplotlib notebook
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import requests
from datetime import datetime
import time
from requests.auth import HTTPBasicAuth
import sys
import io
import datetime
import math 
import sklearn 

In [9]:
def query_opendota(sql):
    resp = requests.get('https://api.opendota.com/api/explorer', params={'sql': sql})
    data = resp.json()
    if resp.status_code == 400 and data is not None:
        sys.stderr.write(data['err'])
    resp.raise_for_status()
    return pd.DataFrame.from_records(data['rows'])

In [16]:
def query_ward_table():
    df_wards = query_opendota('''
    SELECT
    -- Basic table
    matches.match_id,
    player_matches.account_id,
    teams.team_id,
    matches.leagueid,
    leagues.name leaguename,
    matches.start_time,
    player_matches.hero_id,
    player_matches.player_slot,
    ((player_matches.player_slot < 128) = matches.radiant_win) win,
        player_matches.kills,
    player_matches.deaths,
    player_matches.assists,
    player_matches.gold_per_min ,
    player_matches.xp_per_min ,
    player_matches.gold_spent ,
    player_matches.hero_damage ,
    player_matches.tower_damage ,
    player_matches.stuns ,
    player_matches.creeps_stacked ,
    player_matches.camps_stacked ,
    player_matches.hero_healing ,
    player_matches.last_hits,
    player_matches.denies,
    player_matches.level,
    matches.duration,
    matches.cluster,
    matches.first_blood_time,

    --timeStamp variables
    player_matches.gold_t[3] t_Gold_cnt_3,
    player_matches.gold_t[5] t_Gold_cnt_5,
    player_matches.gold_t[8] t_Gold_cnt_8,
    player_matches.gold_t[10] t_Gold_cnt_10,
    player_matches.gold_t[12] t_Gold_cnt_12,
    player_matches.gold_t[15] t_Gold_cnt_15,
    player_matches.gold_t[20] t_Gold_cnt_20,
    player_matches.gold_t[25] t_Gold_cnt_25,
    player_matches.gold_t[30] t_Gold_cnt_30,

    player_matches.lh_t[3] t_LastHits_cnt_3,
    player_matches.lh_t[5] t_LastHits_cnt_5,
    player_matches.lh_t[8] t_LastHits_cnt_8,
    player_matches.lh_t[10] t_LastHits_cnt_10,
    player_matches.lh_t[12] t_LastHits_cnt_12,
    player_matches.lh_t[15] t_LastHits_cnt_15,
    player_matches.lh_t[20] t_LastHits_cnt_20,
    player_matches.lh_t[25] t_LastHits_cnt_25,
    player_matches.lh_t[30] t_LastHits_cnt_30,

    player_matches.xp_t[3] t_Experience_cnt_3,
    player_matches.xp_t[5] t_Experience_cnt_5,
    player_matches.xp_t[8] t_Experience_cnt_8,
    player_matches.xp_t[10] t_Experience_cnt_10,
    player_matches.xp_t[12] t_Experience_cnt_12,
    player_matches.xp_t[15] t_Experience_cnt_15,
    player_matches.xp_t[20] t_Experience_cnt_20,
    player_matches.xp_t[25] t_Experience_cnt_25,
    player_matches.xp_t[30] t_Experience_cnt_30,

    player_matches.pings ping_log,
    player_matches.obs_log,
    player_matches.sen_log,
    player_matches.runes_log,
    player_matches.kills_log,
    player_matches.buyback_log,

    --table_heroStats
    heroes.localized_name,
    heroes.attack_type,
    heroes.primary_attr,
    heroes.roles role_log,

    --purchase_log
    player_matches.purchase_log
    FROM matches
    JOIN match_patch using(match_id)
    JOIN leagues using(leagueid)
    JOIN player_matches using(match_id)
    JOIN heroes on heroes.id = player_matches.hero_id
    LEFT JOIN notable_players ON notable_players.account_id = player_matches.account_id AND notable_players.locked_until = (SELECT MAX(locked_until) FROM notable_players)
    LEFT JOIN teams using(team_id)
    WHERE TRUE
    AND matches.leagueid = 5401
    AND matches.start_time >= extract(epoch from timestamp '2017-08-01T21:00:00.000Z')
    AND teams.team_id IN (5, 15, 39, 46, 2163, 350190, 1375614, 1838315, 1883502, 2108395, 2512249, 2581813, 2586976, 2640025, 2672298, 1333179, 3331948, 1846548)
    ORDER BY matches.match_id DESC NULLS LAST
    LIMIT 10000
    '''.format(**locals()))
    return df_wards

In [23]:
matches = query_ward_table()

In [24]:
matches['id'] = matches['match_id'].astype('str') + "_" + matches['account_id'].astype('str')
print(matches.shape)
matches['datetime'] = matches["start_time"].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%d.%m.%Y %H:%M:%S'))

(1780, 66)


In [25]:
list(matches)

['account_id',
 'assists',
 'attack_type',
 'buyback_log',
 'camps_stacked',
 'cluster',
 'creeps_stacked',
 'deaths',
 'denies',
 'duration',
 'first_blood_time',
 'gold_per_min',
 'gold_spent',
 'hero_damage',
 'hero_healing',
 'hero_id',
 'kills',
 'kills_log',
 'last_hits',
 'leagueid',
 'leaguename',
 'level',
 'localized_name',
 'match_id',
 'obs_log',
 'ping_log',
 'player_slot',
 'primary_attr',
 'purchase_log',
 'role_log',
 'runes_log',
 'sen_log',
 'start_time',
 'stuns',
 't_experience_cnt_10',
 't_experience_cnt_12',
 't_experience_cnt_15',
 't_experience_cnt_20',
 't_experience_cnt_25',
 't_experience_cnt_3',
 't_experience_cnt_30',
 't_experience_cnt_5',
 't_experience_cnt_8',
 't_gold_cnt_10',
 't_gold_cnt_12',
 't_gold_cnt_15',
 't_gold_cnt_20',
 't_gold_cnt_25',
 't_gold_cnt_3',
 't_gold_cnt_30',
 't_gold_cnt_5',
 't_gold_cnt_8',
 't_lasthits_cnt_10',
 't_lasthits_cnt_12',
 't_lasthits_cnt_15',
 't_lasthits_cnt_20',
 't_lasthits_cnt_25',
 't_lasthits_cnt_3',
 't_lasth

In [26]:
resp = requests.get('https://api.opendota.com/api/heroStats')
data = resp.json()
if resp.status_code == 400 and data is not None:
    sys.stderr.write(data['err'])
resp.raise_for_status()

In [27]:
data

[{'1000_pick': 8115,
  '1000_win': 3880,
  '2000_pick': 19628,
  '2000_win': 9778,
  '3000_pick': 24748,
  '3000_win': 12525,
  '4000_pick': 7835,
  '4000_win': 3898,
  '5000_pick': 940,
  '5000_win': 448,
  'agi_gain': 2.8,
  'attack_range': 150,
  'attack_rate': 1.45,
  'attack_type': 'Melee',
  'base_agi': 22,
  'base_armor': -1,
  'base_attack_max': 31,
  'base_attack_min': 27,
  'base_health': 200,
  'base_health_regen': 0,
  'base_int': 15,
  'base_mana': 75,
  'base_mana_regen': 0.01,
  'base_mr': 25,
  'base_str': 22,
  'cm_enabled': True,
  'hero_id': 1,
  'icon': '/apps/dota2/images/heroes/antimage_icon.png',
  'id': 1,
  'img': '/apps/dota2/images/heroes/antimage_full.png?',
  'int_gain': 1.8,
  'legs': 2,
  'localized_name': 'Anti-Mage',
  'move_speed': 315,
  'name': 'npc_dota_hero_antimage',
  'primary_attr': 'agi',
  'pro_ban': 68,
  'pro_pick': 38,
  'pro_win': 18,
  'projectile_speed': 0,
  'roles': ['Carry', 'Escape', 'Nuker'],
  'str_gain': 1.5,
  'turn_rate': 0.5},


In [28]:
def download_google_spreadsheet(url):
    try:
        resp = requests.get(url)
        resp.raise_for_status()
        content_bytes = resp.content
    except requests.exceptions.HTTPError as err:
        print(err, file=sys.stderr)
    try:
        content = pd.read_csv(io.BytesIO(content_bytes))
        return content
    except Exception as err:
        print("failed to convert bytes to csv:", err, file=sys.stderr)

In [29]:
download_google_spreadsheet('https://docs.google.com/spreadsheets/d/e/2PACX-1vRV7SmVCA3a-IgxDRmhEJjFwLner7tEgRXHy_gcgav7GF2pCv4bOlwwt6B8RHkBP8LxV6Idf3zVVnze/pub?output=csv')

,team,player,role,account_id
0,OG,n0tail,1,19672354
1,OG,ana,2,311360822
2,OG,JerAx,4,26771994
3,OG,s4,3,41231571
4,OG,Fly,5,94155156
5,Evil Geniuses,Universe,3,87276347
6,Evil Geniuses,zai,4,73562326
7,Evil Geniuses,Arteezy,1,86745912
8,Evil Geniuses,Suma1l,2,111620041
9,Evil Geniuses,Cr1t,5,25907144
